# Non-standard amino acids in KLIFS molecules

Aims of this notebook:
1. For all KLIFS molecules, save all non-standard amino acids.
2. Check amount of non-standard amino acids. Some `kinsim_structure` features are only defined for standard amino acids, thus we need to check how much information we loose in our dataset.

## Imports and functions

In [1]:
import glob
from pathlib import Path
import pickle
import sys

import pandas as pd

sys.path.extend(['./..'])
from kinsim_structure.auxiliary import read_mol2

In [2]:
def get_non_standard_amino_acids_from_klifs(mol2_path):
    """
    For a given set of mol2 files, collect all non-standard amino acids.

    Parameters
    ----------
    mol2_path : pathlib.Path or str
        Path to mol2 files (can contain wildcards to fetch muliple files at once).

    Returns
    -------
    dict
        List of non-standard amino acids (value) for each structure contained in the input mol2 file (key).
    """

    # Input parameters
    mol2_path = str(mol2_path)  # Needs to be str because of glob and biopandas

    # One or multiple mol2 file paths
    paths = glob.glob(mol2_path)

    # 20 standard amino acids
    standard_aminoacids = 'ALA ARG ASN ASP CYS GLN GLU GLY HIS ILE LEU LYS MET PHE PRO SER THR TRP TYR VAL'.split()

    # Will contain per structure (key) list of non-standard amino acids (values)
    non_standard_aminoacids = {}

    for path in paths:

        molecule = read_mol2(path)
        print(molecule.code)

        # Get deduplicated amino acid names
        subst_names = set([i[:3] for i in set(molecule.df.subst_name)])

        # Retain only non-standard amino acids
        non_standard_aminoacids_per_file = [subst_name for subst_name in subst_names if subst_name not in standard_aminoacids]

        # If structure contains non-standard amino acids, add to dict
        if non_standard_aminoacids_per_file:
            non_standard_aminoacids[molecule.code] = non_standard_aminoacids_per_file

    return non_standard_aminoacids

## IO paths

In [3]:
mol2_path = '/home/dominique/Documents/data/kinsim/20190724_full/raw/KLIFS_download/HUMAN/*/*/pocket.mol2'
output_path = Path('/home/dominique/Documents/projects/kinsim_structure') / 'results' / '20190724_full' / 'non_standard_aminoacids.p'    

## Data generation

In [4]:
non_standard_aminoacids = get_non_standard_amino_acids_from_klifs(mol2_path)

HUMAN/CDK8_3rgf_altA_chainA
HUMAN/CDK8_4f7l_chainA
HUMAN/CDK8_5fgk_altA_chainA
HUMAN/CDK8_4f7j_altB_chainA
HUMAN/CDK8_3rgf_altB_chainA
HUMAN/CDK8_5icp_altA_chainA
HUMAN/CDK8_5hnb_altA_chainA
HUMAN/CDK8_5hbj_altA_chainA
HUMAN/CDK8_5hnb_altB_chainA
HUMAN/CDK8_4f6u_altB_chainA
HUMAN/CDK8_5hvy_altB_chainA
HUMAN/CDK8_5bnj_chainA
HUMAN/CDK8_4f7j_altA_chainA
HUMAN/CDK8_5idn_altA_chainA
HUMAN/CDK8_5fgk_altB_chainA
HUMAN/CDK8_4f7s_altB_chainA
HUMAN/CDK8_5hbe_altB_chainA
HUMAN/CDK8_4f6s_chainA
HUMAN/CDK8_4g6l_chainA
HUMAN/CDK8_5hbj_altB_chainA
HUMAN/CDK8_5hbh_chainA
HUMAN/CDK8_5hvy_altA_chainA
HUMAN/CDK8_5hbe_altA_chainA
HUMAN/CDK8_5i5z_chainA
HUMAN/CDK8_5idn_altB_chainA
HUMAN/CDK8_4f7n_chainA
HUMAN/CDK8_5idp_altA_chainA
HUMAN/CDK8_4crl_chainA
HUMAN/CDK8_4f7s_altA_chainA
HUMAN/CDK8_5xqx_chainA
HUMAN/CDK8_5xs2_altB_chainA
HUMAN/CDK8_5icp_altB_chainA
HUMAN/CDK8_5xs2_altA_chainA
HUMAN/CDK8_5cei_altA_chainA
HUMAN/CDK8_5idp_altB_chainA
HUMAN/CDK8_4f6u_altA_chainA
HUMAN/CDK8_4f6w_chainA
HUMAN/CDK8_5ce

HUMAN/p110d_5dxu_chainA
HUMAN/p110d_6g6w_altA_chainA
HUMAN/CDK2_3ezr_chainA
HUMAN/CDK2_2bhh_chainA
HUMAN/CDK2_6guh_altB_chainA
HUMAN/CDK2_3my5_altC_chainC
HUMAN/CDK2_6q4c_altB_chainA
HUMAN/CDK2_2w05_altA_chainA
HUMAN/CDK2_3r9n_altB_chainA
HUMAN/CDK2_2c68_chainA
HUMAN/CDK2_1urc_altB_chainC
HUMAN/CDK2_2c5x_altB_chainC
HUMAN/CDK2_6q4i_altA_chainA
HUMAN/CDK2_1gz8_altA_chainA
HUMAN/CDK2_4eoj_altB_chainC
HUMAN/CDK2_3my5_altA_chainA
HUMAN/CDK2_6q4j_altB_chainA
HUMAN/CDK2_4ek5_altA_chainA
HUMAN/CDK2_2uue_chainC
HUMAN/CDK2_4fko_altA_chainA
HUMAN/CDK2_2r3l_altB_chainA
HUMAN/CDK2_3r8m_altA_chainA
HUMAN/CDK2_1h0v_altB_chainA
HUMAN/CDK2_1oiy_altA_chainA
HUMAN/CDK2_4cfv_altA_chainA
HUMAN/CDK2_4eos_altB_chainA
HUMAN/CDK2_3pxr_altA_chainA
HUMAN/CDK2_4cfn_altB_chainC
HUMAN/CDK2_3sqq_altB_chainA
HUMAN/CDK2_3pxr_altB_chainA
HUMAN/CDK2_3qx2_altA_chainA
HUMAN/CDK2_1e9h_chainA
HUMAN/CDK2_4cfx_altA_chainC
HUMAN/CDK2_5iex_altB_chainA
HUMAN/CDK2_3pxf_altA_chainA
HUMAN/CDK2_2vtp_altB_chainA
HUMAN/CDK2_1pkd_chai

HUMAN/CDK2_1ol1_chainA
HUMAN/CDK2_6q4d_altC_chainA
HUMAN/CDK2_4ez7_chainA
HUMAN/CDK2_3ej1_chainC
HUMAN/CDK2_1h00_altA_chainA
HUMAN/CDK2_1h1p_chainC
HUMAN/CDK2_4fkq_altB_chainA
HUMAN/CDK2_1pxo_altB_chainA
HUMAN/CDK2_3r1s_altB_chainA
HUMAN/CDK2_1pxn_altA_chainA
HUMAN/CDK2_3bht_altA_chainC
HUMAN/CDK2_3qtq_altB_chainA
HUMAN/CDK2_2iw9_altB_chainC
HUMAN/CDK2_4eor_altB_chainC
HUMAN/CDK2_3rzb_altB_chainA
HUMAN/CDK2_1ol2_chainC
HUMAN/CDK2_1h0w_altB_chainA
HUMAN/CDK2_6q4f_altA_chainA
HUMAN/CDK2_3bhu_altA_chainA
HUMAN/CDK2_4cfv_altB_chainA
HUMAN/CDK2_1ckp_altA_chainA
HUMAN/CDK2_3unj_chainA
HUMAN/CDK2_4eoq_altB_chainA
HUMAN/CDK2_2r3r_altA_chainA
HUMAN/CDK2_4ez3_chainA
HUMAN/CDK2_4cfu_altA_chainA
HUMAN/CDK2_4eon_altA_chainA
HUMAN/CDK2_1h1q_chainA
HUMAN/CDK2_6q4a_altC_chainA
HUMAN/CDK2_1gii_chainA
HUMAN/CDK2_4bcq_altB_chainA
HUMAN/CDK2_3rmf_altB_chainA
HUMAN/CDK2_1ogu_altB_chainC
HUMAN/CDK2_1jst_chainA
HUMAN/CDK2_3qqf_altA_chainA
HUMAN/CDK2_1r78_altB_chainA
HUMAN/CDK2_2ds1_chainA
HUMAN/CDK2_2whb_alt

HUMAN/CDK2_3r73_altB_chainA
HUMAN/CDK2_4eoq_altA_chainC
HUMAN/CDK2_3dog_altA_chainA
HUMAN/CDK2_2r64_chainA
HUMAN/CDK2_5lqe_chainA
HUMAN/CDK2_4ek4_altC_chainA
HUMAN/CDK2_3pxy_altB_chainA
HUMAN/CDK2_2vv9_altA_chainA
HUMAN/CDK2_6q3b_altB_chainA
HUMAN/CDK2_3py0_altB_chainA
HUMAN/CDK2_4bck_altC_chainA
HUMAN/CDK2_2vtp_altA_chainA
HUMAN/CDK2_1pxm_altA_chainA
HUMAN/CDK2_1pf8_chainA
HUMAN/CDK2_1jst_chainC
HUMAN/CDK2_1pxo_altA_chainA
HUMAN/CDK2_5if1_altB_chainA
HUMAN/CDK2_2r3g_altA_chainA
HUMAN/CDK2_4eon_altA_chainC
HUMAN/CDK2_1hcl_altB_chainA
HUMAN/CDK2_2r3i_altC_chainA
HUMAN/CDK2_3r7v_altB_chainA
HUMAN/CDK2_3rni_altA_chainA
HUMAN/CDK2_5iev_altA_chainA
HUMAN/CDK2_1pxi_altA_chainA
HUMAN/CDK2_3qqh_altA_chainA
HUMAN/CDK2_4eol_altB_chainA
HUMAN/CDK2_3ezv_chainA
HUMAN/CDK2_3sw4_altA_chainA
HUMAN/CDK2_4eom_altA_chainA
HUMAN/CDK2_3pxz_chainA
HUMAN/CDK2_3lfq_chainA
HUMAN/CDK2_2c5o_altB_chainC
HUMAN/CDK2_1pxo_altD_chainA
HUMAN/CDK2_4fku_altA_chainA
HUMAN/CDK2_2w1h_chainA
HUMAN/CDK2_2r3i_altB_chainA
HUMA

HUMAN/LOK_4aot_altA_chainA
HUMAN/LOK_6gtt_altB_chainA
HUMAN/LOK_6hxf_altB_chainB
HUMAN/LOK_4equ_altB_chainB
HUMAN/LOK_6hxf_altA_chainC
HUMAN/LOK_4equ_altA_chainA
HUMAN/LOK_4usd_altB_chainB
HUMAN/LOK_4usd_altA_chainA
HUMAN/LOK_5ajq_altB_chainA
HUMAN/LOK_5ajq_altB_chainB
HUMAN/LOK_6gtt_altA_chainA
HUMAN/LOK_5owq_altB_chainB
HUMAN/LOK_6eim_altA_chainB
HUMAN/MER_3brb_altB_chainA
HUMAN/MER_3tcp_chainA
HUMAN/MER_4mha_altA_chainA
HUMAN/MER_5tc0_chainA
HUMAN/MER_5u6c_altA_chainB
HUMAN/MER_3bpr_altB_chainC
HUMAN/MER_2p0c_altA_chainA
HUMAN/MER_5k0x_altB_chainB
HUMAN/MER_5k0x_altB_chainA
HUMAN/MER_2p0c_altA_chainB
HUMAN/MER_4mha_altB_chainB
HUMAN/MER_5u6c_altB_chainB
HUMAN/MER_2p0c_altB_chainB
HUMAN/MER_3bpr_altA_chainD
HUMAN/MER_5u6c_altA_chainA
HUMAN/MER_3tcp_chainB
HUMAN/MER_4m3q_chainA
HUMAN/MER_4mha_altB_chainA
HUMAN/MER_3bpr_altB_chainD
HUMAN/MER_5td2_altA_chainA
HUMAN/MER_3brb_altA_chainA
HUMAN/MER_4mh7_chainB
HUMAN/MER_4mh7_chainA
HUMAN/MER_5td2_altA_chainB
HUMAN/MER_5k0k_altA_chainA
HUMA

...

HUMAN/JAK1_6n7a_altA_chainB
HUMAN/JAK1_4e4l_altA_chainD
HUMAN/JAK1_6elr_altB_chainA
HUMAN/JAK1_6aah_altA_chainA
HUMAN/JAK1_4i5c_altA_chainA
HUMAN/JAK1_6n7d_altB_chainA
HUMAN/JAK1_4ivd_altB_chainB
HUMAN/JAK1_6dbn_altA_chainA
HUMAN/JAK1_6n7a_altB_chainB
HUMAN/JAK1_4e5w_altA_chainB
HUMAN/JAK1_6c7y_altB_chainA
HUMAN/JAK1_4e4l_altB_chainE
HUMAN/JAK1_4e4l_altA_chainB
HUMAN/JAK1_4e5w_altB_chainB
HUMAN/JAK1_4fk6_altB_chainB
HUMAN/JAK1_4ivc_chainA
HUMAN/JAK1_4fk6_altA_chainA
HUMAN/JAK1_4e5w_altB_chainA
HUMAN/JAK1_6n78_altB_chainA
HUMAN/JAK1_6n78_altA_chainA
HUMAN/JAK1_6n7a_altA_chainA
HUMAN/JAK1_5khw_chainB
HUMAN/ALK2_6gin_altB_chainB
HUMAN/ALK2_6gin_altA_chainA
HUMAN/ALK2_3mtf_altB_chainA
HUMAN/ALK2_4bgg_altA_chainB
HUMAN/ALK2_3q4u_altA_chainB
HUMAN/ALK2_3q4u_altB_chainB
HUMAN/ALK2_4bgg_altB_chainD
HUMAN/ALK2_6acr_chainA
HUMAN/ALK2_5oxg_altA_chainC
HUMAN/ALK2_6eix_altA_chainA
HUMAN/ALK2_5oxg_altB_chainC
HUMAN/ALK2_4bgg_altB_chainA
HUMAN/ALK2_5oxg_altA_chainA
HUMAN/ALK2_6gip_altB_chainA
HUMAN/A

HUMAN/ALK_4ctc_chainA
HUMAN/ALK_5aa8_altB_chainA
HUMAN/ALK_4z55_altB_chainA
HUMAN/ALK_2yfx_chainA
HUMAN/ALK_4mkc_altA_chainA
HUMAN/ALK_5ftq_altA_chainA
HUMAN/ALK_5j7h_altB_chainA
HUMAN/ALK_3aox_altB_chainA
HUMAN/ALK_5ftq_altB_chainA
HUMAN/ALK_4fny_chainA
HUMAN/ALK_4anq_chainA
HUMAN/ALK_4fob_chainA
HUMAN/ALK_2xp2_chainA
HUMAN/ALK_4cli_chainA
HUMAN/ALK_4fnw_chainA
HUMAN/ALK_6mx8_altA_chainA
HUMAN/ALK_3lct_altA_chainA
HUMAN/ALK_6e0r_altA_chainA
HUMAN/ALK_3l9p_altA_chainA
HUMAN/ULK3_6fdy_chainU
HUMAN/ULK3_6fdz_chainU
HUMAN/INSR_2b4s_chainB
HUMAN/INSR_5e1s_altB_chainA
HUMAN/INSR_3bu3_chainA
HUMAN/INSR_5hhw_altB_chainA
HUMAN/INSR_1gag_chainA
HUMAN/INSR_2b4s_chainD
HUMAN/INSR_1ir3_chainA
HUMAN/INSR_4ibm_altB_chainA
HUMAN/INSR_5e1s_altA_chainA
HUMAN/INSR_3ekn_altA_chainA
HUMAN/INSR_3ekn_altB_chainA
HUMAN/INSR_3ekk_altA_chainA
HUMAN/INSR_3eta_altB_chainB
HUMAN/INSR_4ibm_altA_chainA
HUMAN/INSR_3eta_altB_chainA
HUMAN/INSR_2z8c_chainA
HUMAN/INSR_1irk_altB_chainA
HUMAN/INSR_2auh_chainA
HUMAN/INSR_3

HUMAN/HH498_6b5j_altA_chainA
HUMAN/HH498_4yfi_altB_chainA
HUMAN/HH498_4yff_altA_chainB
HUMAN/HH498_6b5j_altC_chainC
HUMAN/HH498_4yfi_altA_chainB
HUMAN/HH498_4yfi_altB_chainD
HUMAN/HH498_4yfi_altA_chainA
HUMAN/HH498_4yff_altB_chainD
HUMAN/HH498_4yff_altA_chainD
HUMAN/HH498_4yff_altB_chainA
HUMAN/HH498_6b5j_altB_chainD
HUMAN/HH498_6b5j_altB_chainC
HUMAN/HH498_4yff_altA_chainC
HUMAN/HH498_6b5j_altC_chainD
HUMAN/HH498_6b5j_altA_chainB
HUMAN/HH498_4yfi_altB_chainC
HUMAN/HH498_4yff_altA_chainA
HUMAN/HH498_4yfi_altA_chainD
HUMAN/HH498_6b5j_altB_chainB
HUMAN/HH498_4yff_altB_chainB
HUMAN/HH498_6b5j_altA_chainD
HUMAN/BIKE_5ikw_chainA
HUMAN/BIKE_5i3o_chainB
HUMAN/BIKE_5i3r_chainA
HUMAN/BIKE_4w9w_altB_chainA
HUMAN/BIKE_5i3o_chainA
HUMAN/BIKE_4w9w_altA_chainA
HUMAN/BIKE_5i3r_chainB
HUMAN/BIKE_4w9x_chainA
HUMAN/PKN1_4otg_altB_chainA
HUMAN/PKN1_4oth_altB_chainA
HUMAN/PKN1_4otg_altA_chainA
HUMAN/PKN1_4oth_altA_chainA
HUMAN/PKN1_4oti_altA_chainA
HUMAN/PKN1_4oti_altB_chainA
HUMAN/PKN1_4otd_altA_chainA
H

HUMAN/PIM1_5ndt_altA_chainA
HUMAN/PIM1_4gw8_altA_chainA
HUMAN/PIM1_4k0y_altB_chainA
HUMAN/PIM1_5n52_altB_chainA
HUMAN/PIM1_4alu_altB_chainA
HUMAN/PIM1_5kgk_chainA
HUMAN/PIM1_5tel_altB_chainA
HUMAN/PIM1_1xr1_chainA
HUMAN/PIM1_3umw_altB_chainA
HUMAN/PIM1_1yxt_altB_chainA
HUMAN/PIM1_1xws_altB_chainA
HUMAN/PIM1_1yxu_chainC
HUMAN/PIM1_5tex_chainA
HUMAN/PIM1_4k0y_altA_chainA
HUMAN/PIM1_5n4u_altA_chainA
HUMAN/PIM1_2xix_altA_chainA
HUMAN/PIM1_2bzj_altB_chainA
HUMAN/PIM1_5n4r_altB_chainA
HUMAN/PIM1_5kgi_chainA
HUMAN/PIM1_4as0_altA_chainA
HUMAN/PIM1_2xj1_altA_chainA
HUMAN/PIM1_3vbq_altB_chainA
HUMAN/PIM1_6mt0_chainA
HUMAN/PIM1_6no9_chainA
HUMAN/PIM1_5v82_chainA
HUMAN/PIM1_5n4u_altB_chainA
HUMAN/PIM1_3vbv_chainA
HUMAN/PIM1_5kcx_altB_chainA
HUMAN/PIM1_3r04_altB_chainA
HUMAN/PIM1_5n50_altB_chainA
HUMAN/PIM1_4gw8_altB_chainA
HUMAN/PIM1_1xqz_chainA
HUMAN/PIM1_5n4x_altA_chainA
HUMAN/PIM1_4jx3_chainA
HUMAN/PIM1_2xiz_altB_chainA
HUMAN/PIM1_2c3i_altB_chainB
HUMAN/PIM1_5vub_altA_chainB
HUMAN/PIM1_3bgz_cha

HUMAN/BTK_3piz_chainA
HUMAN/BTK_5kup_altA_chainA
HUMAN/BTK_5vfi_altB_chainA
HUMAN/BTK_4rg0_chainA
HUMAN/BTK_5p9m_altC_chainA
HUMAN/BTK_4nwm_altA_chainA
HUMAN/BTK_5fbn_altB_chainD
HUMAN/BTK_6di5_altA_chainA
HUMAN/BTK_3p08_chainB
HUMAN/BTK_5bpy_chainB
HUMAN/BTK_5vgo_altB_chainA
HUMAN/BTK_6nfi_altA_chainA
HUMAN/BTK_3piy_chainA
HUMAN/BTK_6o8i_altB_chainA
HUMAN/BTK_5xyz_altB_chainB
HUMAN/BTK_6bke_altB_chainA
HUMAN/BTK_6hrp_chainA
HUMAN/BTK_5j87_altB_chainA
HUMAN/BTK_6bln_altA_chainA
HUMAN/BTK_5bpy_chainA
HUMAN/BTK_5fbn_altA_chainD
HUMAN/BTK_5j87_altB_chainC
HUMAN/BTK_4ot6_chainA
HUMAN/BTK_5p9j_altB_chainA
HUMAN/BTK_4yhf_altA_chainB
HUMAN/BTK_5p9h_altA_chainA
HUMAN/BTK_5p9f_altA_chainA
HUMAN/BTK_6bkw_altA_chainA
HUMAN/BTK_5j87_altA_chainD
HUMAN/BTK_5zz4_chainC
HUMAN/BTK_5j87_altA_chainB
HUMAN/BTK_4yhf_altB_chainA
HUMAN/BTK_5p9k_altB_chainA
HUMAN/BTK_6di1_altB_chainA
HUMAN/BTK_5zz4_chainD
HUMAN/BTK_5j87_altB_chainD
HUMAN/BTK_4zly_altC_chainA
HUMAN/BTK_5j87_altA_chainC
HUMAN/BTK_6di1_altA_chai

In [5]:
with open(output_path, 'wb') as f:
    pickle.dump(non_standard_aminoacids, f)

## Data analysis

In [6]:
with open(output_path, 'rb') as f:
    non_standard_aminoacids = pickle.load(f)

In [27]:
print(f'Number of non-standard amino acids in KLIFS dataset: {len(non_standard_aminoacids)}')

Number of non-standard amino acids in KLIFS dataset: 30


In [12]:
pd.Series(non_standard_aminoacids)

HUMAN/CDK2_2cjm_chainA             [PTR]
HUMAN/CDK2_1oir_chainA             [KCX]
HUMAN/CDK2_2cjm_chainC             [PTR]
HUMAN/AurA_4j8m_chainA             [CAF]
HUMAN/SRC_1yi6_chainB              [PTR]
HUMAN/SRC_1yi6_chainA              [PTR]
HUMAN/p38a_3d83_chainA             [CSO]
HUMAN/KDR_3c7q_chainA              [CME]
HUMAN/EGFR_2j5e_chainA             [CY0]
HUMAN/CHK1_2ydj_chainA             [CSS]
HUMAN/CHK1_2ydj_chainB             [CSS]
HUMAN/IRAK4_2o8y_chainA            [MSE]
HUMAN/IRAK4_2o8y_chainB            [MSE]
HUMAN/DNAPK_5luq_chainB            [MSE]
HUMAN/DNAPK_5luq_chainA            [MSE]
HUMAN/MAPKAPK2_1nxk_chainC         [MSE]
HUMAN/MAPKAPK2_1nxk_chainA         [MSE]
HUMAN/MAPKAPK2_1nxk_chainD         [MSE]
HUMAN/MAPKAPK2_1nxk_chainB         [MSE]
HUMAN/JNK3_2r9s_chainB             [OCY]
HUMAN/JNK3_2r9s_chainA             [OCY]
HUMAN/RET_5fm2_chainA              [PTR]
HUMAN/RET_5fm3_chainA              [PTR]
HUMAN/ILK_3kmu_chainA              [MSE]
HUMAN/RIOK1_4otp

In [29]:
# Get set of non-standard amino acids
non_standards = []

for k, v in non_standard_aminoacids.items():
    non_standards = non_standards + v
    
non_standards_set = set(non_standards)

In [30]:
non_standards_set

{'CAF', 'CME', 'CSO', 'CSS', 'CY0', 'KCX', 'MSE', 'OCY', 'PHD', 'PTR'}

In [25]:
smiles = {
    'CAF': 'C[As](C)(=O)SC[C@H](N)C(O)=O',
    'CME': 'N[C@@H](CSSCCO)C(O)=O',
    'CSO': 'N[C@@H](CSO)C(O)=O',
    'CSS': 'N[C@@H](CSS)C(O)=O',
    'CY0': 'N[C@@H](CSCCC(=O)Nc1ccc2ncnc(Nc3ccccc3)c2c1)C(O)=O',
    'KCX': 'N[C@@H](CCCCNC(O)=O)C(O)=O',
    'MSE': 'C[Se]CC[C@H](N)C(O)=O',
    'OCY': 'N[C@@H](CSCCO)C(O)=O',
    'PHD': 'N[C@@H](CC(=O)OP(O)(O)=O)C(O)=O',
    'PTR': 'N[C@@H](Cc1ccc(OP(O)(O)=O)cc1)C(O)=O',
}

Check these non-standard residues in the PDB:
* CAF, CME, CSO, CSS, OCY: L-peptide linking (parent CYS)
* KCX: L-peptide linking (parent LYS)
* MSE: L-peptide linking (parent MET) - SELENOMETHIONINE
* PHD: L-peptide linking (parent ASP) - ASPARTYL PHOSPHATE
* PTR: L-peptide linking (parent TYR) - O-PHOSPHOTYROSINE

Additional phosphorylated residues:
* SEP: L-peptide linking (parent SER) - PHOSPHOSERINE

Include phosphorylated amino acids?

In [26]:
phos_aminoacids = 'MSE PHD PTR SEP'.split()